# Installing the basic packages needed for Fine Tuning the BERT model

In [1]:
!pip install --upgrade datasets fsspec transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.6/503.6 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.3/199.3 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 119.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 62.0 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.56.1
    Uninstalling transformers-4.56.1:
      Successfully uninstalled transformers-4.56.1
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling da

In [2]:
from datasets import load_dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

In [3]:
dataset = load_dataset("imdb")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [4]:
train_dataset = dataset["train"].select(range(1000))
test_dataset = dataset["test"].select(range(500))

In [5]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [6]:
# Creating a tokenizer function

def tokenizer_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=256)

In [7]:
# Applying tokenization and formatting the data in a single flow

def preprocess(ds):
  ds = ds.map(tokenizer_function, batched=True, remove_columns=["text"])
  ds = ds.rename_column("label", "labels")
  ds.set_format(type = "torch", columns=["input_ids", "attention_mask", "labels"])
  return ds

In [8]:
train_dataset = preprocess(train_dataset)
test_dataset = preprocess(test_dataset)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [9]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
training_args = TrainingArguments(
    output_dir = "./bert-finetuned-imdb",
    num_train_epochs = 1,
    per_device_train_batch_size = 8,
    logging_dir = "./logs",
    learning_rate = 2e-5,
    weight_decay = 0.01,
    report_to = "none"
)

In [13]:
# Trainer

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = test_dataset,
)

In [14]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=125, training_loss=0.02225022506713867, metrics={'train_runtime': 13.2684, 'train_samples_per_second': 75.367, 'train_steps_per_second': 9.421, 'total_flos': 131555527680000.0, 'train_loss': 0.02225022506713867, 'epoch': 1.0})

In [15]:
trainer.save_model("./bert-finetuned-imdb")

In [19]:
tokenizer.save_pretrained("./bert-finetuned-imdb")

('./bert-finetuned-imdb/tokenizer_config.json',
 './bert-finetuned-imdb/special_tokens_map.json',
 './bert-finetuned-imdb/vocab.txt',
 './bert-finetuned-imdb/added_tokens.json')

In [16]:
!tensorboard --logdir ./logs

/usr/local/lib/python3.12/dist-packages/tensorboard/_vendor/bleach/sanitizer.py:292: SyntaxWarning: invalid escape sequence '\s'
  "[`\000-\040\177-\240\s]+",
/usr/local/lib/python3.12/dist-packages/tensorboard/_vendor/bleach/sanitizer.py:339: SyntaxWarning: invalid escape sequence '\s'
  style = re.compile('url\s*\(\s*[^\s)]+?\s*\)\s*').sub(' ', style)
/usr/local/lib/python3.12/dist-packages/tensorboard/_vendor/bleach/sanitizer.py:354: SyntaxWarning: invalid escape sequence '\s'
  if not re.match("^\s*([-\w]+\s*:[^:;]*(;\s*|$))*$", style):
/usr/local/lib/python3.12/dist-packages/tensorboard/_vendor/bleach/sanitizer.py:358: SyntaxWarning: invalid escape sequence '\w'
  for prop, value in re.findall('([-\w]+)\s*:\s*([^:;]*)', style):
2025-10-02 21:25:23.255716: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759440323.276737   10736 cuda_dnn.

In [17]:
# Evaluating the model

metrics = trainer.evaluate()

In [18]:
print(metrics)

{'eval_loss': 0.0009768720483407378, 'eval_runtime': 1.7829, 'eval_samples_per_second': 280.438, 'eval_steps_per_second': 35.335, 'epoch': 1.0}


# To make a prediction with the Fine Tuned Model

In [20]:
tokenizer = BertTokenizer.from_pretrained("./bert-finetuned-imdb")
model = BertForSequenceClassification.from_pretrained("./bert-finetuned-imdb")

In [21]:
from transformers import pipeline
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)

Device set to use cuda:0


In [31]:
text = "This movie was good!"
result = classifier(text)

In [32]:
print(result)

[{'label': 'LABEL_0', 'score': 0.9939365983009338}]


# Pushing the Fine Tuned model to Hugging Face

In [33]:
from huggingface_hub import notebook_login
notebook_login()

In [34]:
from huggingface_hub import whoami
print(whoami())

{'type': 'user', 'id': '671f58d678721464275180e7', 'name': 'AtharvaBOT7', 'fullname': 'Atharva Chundurwar', 'email': 'atharvachundurwar841@gmail.com', 'emailVerified': True, 'canPay': False, 'periodEnd': None, 'isPro': False, 'avatarUrl': 'https://cdn-avatars.huggingface.co/v1/production/uploads/no-auth/p2Tv-NKUAtssx-fz5Q5th.png', 'orgs': [], 'auth': {'type': 'access_token', 'accessToken': {'displayName': 'testingwrite', 'role': 'write', 'createdAt': '2025-10-01T18:54:53.908Z'}}}


In [36]:
tokenizer.push_to_hub("AtharvaBOT7/bert-finetuned-imdb")
model.push_to_hub("AtharvaBOT7/bert-finetuned-imdb")

README.md: 0.00B [00:00, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...ed-imdb/model.safetensors:   0%|          | 14.2kB /  438MB            

CommitInfo(commit_url='https://huggingface.co/AtharvaBOT7/bert-finetuned-imdb/commit/12ed1d5f83d53f071903276a2d796c67ef9a40bf', commit_message='Upload BertForSequenceClassification', commit_description='', oid='12ed1d5f83d53f071903276a2d796c67ef9a40bf', pr_url=None, repo_url=RepoUrl('https://huggingface.co/AtharvaBOT7/bert-finetuned-imdb', endpoint='https://huggingface.co', repo_type='model', repo_id='AtharvaBOT7/bert-finetuned-imdb'), pr_revision=None, pr_num=None)

# Fine Tuning the Base BERT Model for different tasks

In [37]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import (
    BertTokenizer,
    BertForSequenceClassification,
    BertForTokenClassification,
    BertForQuestionAnswering,
    get_linear_schedule_with_warmup
)

from datasets import load_dataset
from sklearn.metrics import accuracy_score, classification_report, f1_score
import numpy as np
from tqdm import tqdm
from torch.optim import AdamW

In [38]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda
